# Functional Data Structures

A **Functional data structure** is operated on using only **pure functions**.
Therefore, functional data structures are by definition **immutable**.

Example:

The empty list (List() or Nil) is as eternal and immutable as the integer values 3 or 4.


## singly linked list

In [2]:

sealed trait List[+A] //형식 A에 대해 매개변수화된 List자료형식
case object Nil extends List[Nothing]
case class Cons[+A](head: A, tail: List[A]) extends List[A]

object List {
    def sum(ints: List[Int]): Int = ints match {
        case Nil => 0
        case Cons(x,xs) => x + sum(xs)
    }
    
    def product(ds: List[Double]): Double = ds match {
        case Nil => 1.0
        case Cons(0.0, _) => 0.0
        case Cons(x,xs) => x * product(xs)
    }
    
    def apply[A](as: A*): List[A] = {
        if(as.isEmpty) Nil
        else Cons(as.head, apply(as.tail: _*))
    }
}

defined trait List
defined object Nil
defined class Cons
defined object List

In [4]:
sealed trait List[+A] //sealed trait 인터페이스처럼 타입정의?
                      //sealed 하나의 파일안에 모든 차일드(상속받아 생성되는)가 정의되고 더 상속받을 수 없음

final case object Nil extends List[Nothing]
final case class Cons[+A](head: A, tail: List[A]) extends List[A]

object List {   //companion object 
    def apply[A](as: A*): List[A] = {
        if(as.isEmpty) Nil
        else Cons(as.head, apply(as.tail: _*))
    }
}

defined trait List
defined object Nil
defined class Cons
defined object List

In [5]:
val ex1: List[Double] = Nil
val ex2: List[Int] = Cons(1, Nil)
val ex3: List[String] = Cons("a", Cons("b", Nil))

ex1: List[Double] = Nil
ex2: List[Int] = Cons(1,Nil)
ex3: List[String] = Cons(a,Cons(b,Nil))

### Variance Annotation

In trait List[+A],
              ^
              variance  annotation indicating that A is a covariant.
              
Covariant: For all types X and Y,
 - if X <: Y, hen List[X] <: List[Y].
 
If not annotated, parameter is *invariant*, meaning there is no subtyping relationship List[X] and List[Y].

Nil extends List[Nothing]. Since Nothing is a subtype of all types, in conjunction with the variance annotation,
 - Nil can be considered a sub type of any List[XXX]

### Companion Object
We'll often declare a companion object in addition to our data type and its data constructors.

The companion object is the one with *the same name* as the data type(in this case List) where we put various convenience functions for creating or working with values of the data type.

```scala
object List {
    def sum(xs: List[Int]): Int = ???
    def product(xs: List[Double]): Double = ???
    def apply[A](as: A*): List[A] = ???
    ...
}
```

### Pattern Matching

```scala
def sum(xs: List[Int]): Int = xs match {
    case Nil => 0
    case Cons(x, xsl) => x + sum(xsl)
}
```

Pattern mathcing descends into the structure of the expression it examinges and extract subexpressions of that structure.
```scala
target match { pattern => result; ...}
```
If the target matches the pattern in a case, the result of that case becomes the result of the entire match expression.

If multiple patterns match the target, Scala chooses the first matching case.

In [9]:
// Pattern Matching Example
List(1,2,3) match { case Cons(h, _) => h}

// Variable pattern, _, matches any expression.
List(1,2,3) match { case _ => 42}

// Data constructor pattern in conjunction with variables to capture 
// or bind a subexpression of the target
List(1,2,3) match { case Cons(_, t) => t}

// List(1,2,3) match { case Nil => 42} 
// scala.MatchError

: 

In [ ]:
// Exercise
what will be 

### Variadic functions

The function apply in the comanion object List is a variadic function, meaning it accepts zero or more arguments of type A:

---
```scala
def apply[A](as: A*): List[A] = 
    if (as.isEmpty) Nil
    else Cons(as.head, apply(as.tail: _*)) // _ 뒤에 *는 시퀀스 타입인 _을 분해해주는 것을 뜻함 
```
---

Variadic functions provides alittle *syntatic sugar* for creating and passing a `Seq` of elemnts explicitly.

## Data Sharing and Persistent Data Structures

```scala
def tail[A](as: List[A]): List[A] = as match {
    case Cons(h, t) => t
    case Nil => throw new UnsupportedOperationException;
}
```

### Efficiency of Data Sharing

**Vector** in Scla standard library is a purely functional sequence implementation
벡터는 각종 오퍼레이션의 시간복잡도가 상수임


append adds all the elements of one list to the end of another:
```scala
def append[A](a1: List[A], a2: List[A]): List[A] = a1 match {
    case Nil => a2
    case Cons(h, t) => Cons(h, append(t, a2))
}
```

In [10]:
def append[A](a1: List[A], a2: List[A]): List[A] = a1 match {
    case Nil => a2
    case Cons(h, t) => Cons(h, append(t, a2))
}

defined function append

In [11]:
append(List(1,2,3), List(4,5))

res10: List[Int] = Cons(1,Cons(2,Cons(3,Cons(4,Cons(5,Nil)))))

Exercise

Implement a function, *init*, htat returns a List consisting of all but the last element of a List.
```scala
init(list(1,2,3,4)) == List(1,2,3)
```

In [12]:
def init[A](as: List[A]): List[A] = as match {
    case Cons(_, Nil) => Nil
    case Cons(h, t) => Cons(h, init(t))
    case Nil => throw new UnsupportedOperationException;
}

defined function init

In [14]:
def dropWhile_[A](l: List[A], f: A => Boolean): List[A] = l match {
    case Cons(h, t) if (f(h)) => dropWhile_(t, f)
    case e @ Cons(h, t) => e
    case _ => Nil
}

defined function dropWhile_

Improving Type inference for HOFs

Use curried form to maximize type inference.

In [15]:
def dropWhile[A](l: List[A])(f: A => Boolean): List[A] = l match {
    case Cons(h, t) if (f(h)) => dropWhile(t)(f)
    case e @ Cons(h, t) => e
    case _ => Nil
}

defined function dropWhile

Don't Repeat Yourself (DRY)

``` scala
def sum(ints: List[Int]): Int = ints match {
    case Nil => 0
    case Cons(x,xs) => x + sum(xs)
}
    
def product(ds: List[Double]): Double = ds match {
    case Nil => 1.0
    case Cons(x,xs) => x * product(xs)
}
```

### foldRight

In [16]:
def foldRight[A,B](as: List[A], z: B)(f: (A, B) => B): B = as match {
    case Nil => z
    case Cons(h,t) => f(h, foldRight(t,z)(f))
}

defined function foldRight

In [17]:
def sum(ns: List[Int]) = 
    foldRight(ns, 0)((x,y) => x+y)
def product(ns: List[Double]) = 
    foldRight(ns, 1.0)(_ * _)

defined function sum
defined function product

### foldLeft

In [18]:
def foldLeft[A,B](as: List[A], z: B)(f: (B, A) => B): B = as match {
    case Cons(h,t) => foldLeft(t, f(z, h))(f)
    case Nil => z
}

defined function foldLeft

### Exercise

In [19]:
def length[A](as: List[A]): Int = 
    foldRight(as, 0)((_, acc) => 1 + acc)

defined function length

In [21]:
// 요거는 안뒤집힘 그대로임 foldRight로 넘겨준 f가 맨끝원소부터 적용되기 때문에 그대로 쌓임
def reverse[A](as: List[A]): List[A] =
    foldRight(as, Nil: List[A])((a, acc) => Cons(a, acc))
// or foldRight(as, List[A]())((a, acc) => Cons(a, acc))
// or foldRight(as, List.empty[A])((a, acc) => Cons(a, acc))

defined function reverse

In [22]:
def reverse[A](as: List[A]): List[A] =
    foldLeft(as, Nil: List[A])((acc, a) => Cons(a, acc))
// or foldLeft(as, List[A]())((acc, a) => Cons(a, acc))
// or foldLeft(as, List.empty[A])((acc, a) => Cons(a, acc))

defined function reverse